In [2]:
from read_data import get_training, get_Doc2Vec
from processing import combine_with_vec, exclude_non_numeric
import pandas as pd
import numpy as np
from sklearn.pipeline import make_pipeline
from sklearn.preprocessing import StandardScaler, FunctionTransformer
from sklearn.compose import make_column_transformer
from sklearn.model_selection import train_test_split, cross_val_score, ShuffleSplit
from sklearn.linear_model import LogisticRegression
np.random.seed(30027)

In [3]:
# Training data
X, y = get_training()
train_name_vec50, train_ingr_vec50, train_steps_vec50 = get_Doc2Vec(data="train", num_features=50)
train_name_vec100, train_ingr_vec100, train_steps_vec100 = get_Doc2Vec(data="train", num_features=100)

In [4]:
# Combine Doc2Vec with 50 features and non-numeric train data
temp_X = X.copy()
temp_X = exclude_non_numeric(temp_X)
temp_train_name_vec50 = train_name_vec50.copy()
temp_train_ingr_vec50 = train_ingr_vec50.copy()
temp_train_steps_vec50 = train_steps_vec50.copy()
X_combined_50: pd.DataFrame = combine_with_vec(temp_X, temp_train_name_vec50, temp_train_ingr_vec50, temp_train_steps_vec50)

In [6]:
X_train_50, X_test_50, y_train_50, y_test_50 = train_test_split(X_combined_50, y, test_size=0.2)

In [5]:
LogisticRegression_pipeline = make_pipeline(
    make_column_transformer((FunctionTransformer(np.log1p), ["n_ingredients", "n_steps"]), remainder='passthrough'),
    StandardScaler(),
    LogisticRegression()
)

In [9]:
%%time
partial_LogisticRegression_50Doc2Vec_model = LogisticRegression_pipeline.fit(X_train_50, y_train_50)

CPU times: user 4.55 s, sys: 733 ms, total: 5.28 s
Wall time: 1.05 s


In [12]:
cross_val_score(LogisticRegression_pipeline, X_test_50, y_test_50, cv=ShuffleSplit(n_splits=5, test_size=0.2))

array([0.715625, 0.701875, 0.72    , 0.69875 , 0.7275  ])

In [13]:
partial_LogisticRegression_50Doc2Vec_model.score(X_test_50, y_test_50)

0.721875